<a href="https://colab.research.google.com/github/IRENGBAM/IRENGBAM/blob/main/cbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model


# importing training data
from google.colab import files
uploaded = files.upload()
training_data = pd.read_csv("training_data.csv")

In [ ]:


# preprocessing training data
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

# preprocessing target variable(tags)
le = LabelEncoder()
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()

# creating DNN
edubot = Sequential()
edubot.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
edubot.add(Dense(8))
edubot.add(Dense(8))
edubot.add(Dense(6))
edubot.add(Dense(len(training_data_tags_dummy_encoded[0]), activation="softmax"))
edubot.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# fitting DNN
edubot.fit(training_data_tfidf, training_data_tags_dummy_encoded, epochs=50, batch_size=32)

# saving model file
save_model(edubot, "EduBot_v2")

In [ ]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import load_model
import json
import random


# importing training data
training_data = pd.read_csv("training_data_EduBot_v2.csv")

# loading model
edubot_v2 = load_model("EduBot_v2")

# importing responses
responses = json.load(open("responses.json", "r"))


# fitting TfIdfVectorizer with training data to preprocess inputs
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
vectorizer.fit(training_data["patterns"])


# fitting LabelEncoder with target variable(tags) for inverse transformation of predictions
le = LabelEncoder()
le.fit(training_data["tags"])


# preprocessing input
def predict_tag(inp_str):
    inp_data_tfidf = vectorizer.transform([inp_str.lower()]).toarray()
    predicted_proba = edubot_v2.predict(inp_data_tfidf)
    encoded_label = [np.argmax(predicted_proba)]
    predicted_tag = le.inverse_transform(encoded_label)[0]
    return predicted_tag


# chat function
def start_chat():
    print("---------------  EduBot_v2 - AI Chat bot  ---------------")
    print("Ask any queries regarding SASTRA...")
    print("I will try to understand you and reply...")
    print("Type EXIT to quit...")
    while True:
        inp = input("Ask anything... : ")
        if inp == "EXIT":
            break
        else:
            if inp:
                tag = predict_tag(inp)
                response = random.choice(responses[tag])
                print("Response... : ", response)
            else:
                pass

# calling chat function to check functionality of the program
start_chat()